In [67]:
# Dependencies
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np
import json
import requests
import time
from us import states

In [68]:
#load data
csv_path = "Resources/NutritionPhysicalActivityObesity.csv"
nutrition_data = pd.read_csv(csv_path)

In [69]:


# drop unesscary columns
nutrition_data = nutrition_data.drop(columns=['Data_Value_Unit',
                                              'Stratification1',
                                             ])
#split lat and long
nutrition_data[['Lat','Long']] = nutrition_data.GeoLocation.str.split(",",expand=True,)
#nutrition_data[['openperen']] = nutrition_data.Lat.str.split('(',expand=False,)
#nutrition_data ['Lat'] = nutrition_data['Lat'].to_numeric()
nutrition_data.dtypes

#clean split columns
cols_to_clean = ['Lat']
#nutrition_data['Lat'] = pd.to_numeric(nutrition_data['Lat'])

cols_to_clean = re.sub("[()]", "", str( cols_to_clean))
#print(cols_to_clean)
nutrition_data.head()
#rename labels
#renamecols_df = nutrition_data.rename(columns={"LocationAbbr":"StateAbbr",
                       #    "LocationDesc":"State",
                       #     "Age(years)":"Age"})

#renamecols is df before narrowing age demographic
#renamecols_df.head()


,YearStart,YearEnd,LocationAbbr,LocationDesc,Class,Topic,Question,Data_Value,Data_Value_Alt,Low_Confidence_Limit,...,Income,Race/Ethnicity,GeoLocation,ClassID,TopicID,QuestionID,LocationID,StratificationCategory1,Lat,Long
0,2012,2012,WY,Wyoming,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,48.5,48.5,32.3,...,NaN,American Indian/Alaska Native,"(43.235541343, -108.109830353)",OWS,OWS1,Q037,56,Race/Ethnicity,(43.235541343,-108.109830353)
1,2012,2012,DC,District of Columbia,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,31.6,31.6,24.0,...,NaN,NaN,"(38.890371385, -77.031961127)",OWS,OWS1,Q036,11,Education,(38.890371385,-77.031961127)
2,2011,2011,AL,Alabama,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,35.2,35.2,30.7,...,NaN,NaN,"(32.840571122, -86.631860762)",OWS,OWS1,Q036,1,Age (years),(32.840571122,-86.631860762)
3,2013,2013,US,National,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,27.9,27.9,27.6,...,NaN,NaN,NaN,PA,PA1,Q047,59,Gender,NaN,NaN
4,2011,2011,US,National,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,16.9,16.9,16.0,...,NaN,NaN,NaN,PA,PA1,Q047,59,Age (years),NaN,NaN


In [136]:
#trim age column to include only desired age demographic
agedem_df = renamecols_df.loc[(renamecols_df["Age"] == "18 - 24")
                               |(renamecols_df["Age"] == "25 - 34")
                               |(renamecols_df["Age"] == "35 - 44")
                               |(renamecols_df["Age"] == "45 - 54"),:]
#age is df with focus age deomographic
#while working on indivudal projects we should create and save a df with the needed data
agedem_df.head()

#define dataframe for veggie and fruit analysis
#define dataframe for excercise/behavior analysis
#define dataframe for obesity rates for gen x and gen y

,StateAbbr,State,Class,Topic,Question,Data_Value_Type,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Age,GeoLocation
2,AL,Alabama,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,Value,35.2,35.2,NaN,NaN,30.7,40.0,598.0,25 - 34,"(32.840571122, -86.631860762)"
4,US,National,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,16.9,16.9,NaN,NaN,16.0,17.8,20923.0,18 - 24,NaN
5,US,National,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,22.1,22.1,NaN,NaN,21.4,22.8,45883.0,25 - 34,NaN
30,NY,New York,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,Value,29.7,29.7,NaN,NaN,26.7,32.8,1331.0,35 - 44,"(42.827001032, -75.543970427)"
32,MT,Montana,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,Value,35.5,35.5,NaN,NaN,31.7,39.5,988.0,35 - 44,"(47.066528972, -109.424420645)"
